In [1]:
SERV_URL = 'http://impersonalis.pythonanywhere.com/'

In [2]:
import requests

#### Проверим работу удалённого сервиса

In [3]:
r = requests.get(SERV_URL+'test')
r.text

"app is run! Server's time is [2021-10-21 22:08:54.497313]"

# Обзор доступных классов моделей

In [4]:
r = requests.get(SERV_URL+'classes')
r.json()

['LogReg', 'RandomForest', 'ExtraTrees']

# Обзор уже имеющихся моделей

In [5]:
r = requests.get(SERV_URL+'models')
r.json()

[{'is_fitted': False,
  'name': 'default_model_for_tests',
  'params': {'random_state': 0},
  'type': 'LogReg',
  'uid': 1}]

# Создание новой модели

In [6]:
r = requests.post(SERV_URL+'models', json={'type': 'LogReg'})
ans = r.json()
my_current_model_id = ans['uid']
print(ans)

{'is_fitted': False, 'name': 'default_model_for_tests', 'params': {'random_state': 0}, 'type': 'LogReg', 'uid': 2}


#### Забыли указать имя (для удобства) и ГП - обновляем параметры

In [7]:
r = requests.put(SERV_URL+'models/100500', json={'name':'my_log_reg_clf', 'params':{'random_state': 42}})
r.json()

###### В случае успеха возвращается новое описание модели. Описания нет, так как мы ошиблись и указали несущетсвующий UID. Попробуем ещё раз

In [8]:
r = requests.put(SERV_URL+'models/'+str(my_current_model_id), json={'name':'my_log_reg_clf', 'params':{'random_state': 42}})
r.json()

{'is_fitted': False,
 'name': 'my_log_reg_clf',
 'params': {'random_state': 42},
 'type': 'LogReg',
 'uid': 2}

# Ещё раз взглянем на список существующих моделей

In [9]:
r = requests.get(SERV_URL+'models')
r.json()

[{'is_fitted': False,
  'name': 'default_model_for_tests',
  'params': {'random_state': 0},
  'type': 'LogReg',
  'uid': 1},
 {'is_fitted': False,
  'name': 'my_log_reg_clf',
  'params': {'random_state': 42},
  'type': 'LogReg',
  'uid': 2}]

# Удалим ненужную модель

In [10]:
r = requests.delete(SERV_URL+'models/1')
print('Удаление успешно?', r.text)
r = requests.get(SERV_URL+'models')
r.json()

Удаление успешно? "True"



[{'is_fitted': False,
  'name': 'my_log_reg_clf',
  'params': {'random_state': 42},
  'type': 'LogReg',
  'uid': 2}]

# Обучим модель

In [11]:
import numpy as np

#### Возьмём тестовый датасет

In [12]:
from sklearn.datasets import load_iris

In [13]:
X, y = load_iris(return_X_y=True)

In [14]:
data = {
    'x':[[float(v) for v in row] for row in X],
    'y':[int(v) for v in y]
}

#### Обучение

In [15]:
r = requests.put(SERV_URL+'models/100500/fit', json=data)
r.text

'Wrong model`s UID (100500)'

###### ой, ошиблись с uid модели

In [16]:
r = requests.put(SERV_URL+'models/'+str(my_current_model_id)+'/fit', json=data)
r.text

'fitted model with uid 2'

#### Посмотрим состояние моделей

In [17]:
r = requests.get(SERV_URL+'models')
r.json()

[{'is_fitted': True,
  'name': 'my_log_reg_clf',
  'params': {'random_state': 42},
  'type': 'LogReg',
  'uid': 2}]

# Применим классификатор

#### Возьмём всё ту же обучающую выборку

In [18]:
X, y = load_iris(return_X_y=True)

In [19]:
data = {
    'x':[[float(v) for v in row] for row in X],
    'proba':int(True)
}

#### Прогноз

In [20]:
r = requests.post(SERV_URL+'models/'+str(my_current_model_id)+'/predict', json=data)

In [21]:
ans = r.json()
preds = np.array(ans['x']).reshape(ans['shape'])
preds[:5]

array([[8.78030305e-01, 1.21958900e-01, 1.07949250e-05],
       [7.97058292e-01, 2.02911413e-01, 3.02949242e-05],
       [8.51997665e-01, 1.47976480e-01, 2.58550858e-05],
       [8.23406019e-01, 1.76536159e-01, 5.78217704e-05],
       [8.96034973e-01, 1.03953836e-01, 1.11907339e-05]])

#### Оценим качество

In [22]:
from sklearn.metrics import accuracy_score

In [23]:
data = {
    'x':[[float(v) for v in row] for row in X],
    'proba':int(False)
}

In [24]:
r = requests.post(SERV_URL+'models/'+str(my_current_model_id)+'/predict', json=data)

In [25]:
ans = r.json()
preds = np.array(ans['x']).reshape(ans['shape'])
preds[:5]

array([0., 0., 0., 0., 0.])

In [26]:
accuracy_score(y, preds)

0.96